<a href="https://colab.research.google.com/github/GParolini/microgrid_model/blob/main/mip_microgrid_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
! pip install mip

In [44]:
# imports
import pandas as pd
from sys import stdout as out
from mip import *

In [45]:
# scalars

time_period = 1               # Length of time period in hours
emission_mac = 0.6038         # Grid emissions
emission_pv = 25.86           # Photovoltaic system lifecycle emissions (per year)
emission_bess = 12.06         # Battery lifecycle emissions (per year)
cost_mac =  0.1176             # Price for one kWh purchased from the grid
cost_pv =  650                # Amortised cost of one kW of installed photovoltaic capacity (per year)
max_pv_cap =  1000            # Maximum PV capacity
cost_bess = 240               # Amortised cost of one kWh of installed battery capacity (per year)
max_bess_cap = 500            # Maximum BESS capacity
F = 0.9                       # Battery charging efficiency factor
D = 0.003                     # Battery self-discharge
g_max = 0.3                   # Battery max rate of charging
g_min = 0.3                   # Battery min rate of discharging

In [46]:
# parameters

dc_load = pd.read_csv("/content/drive/MyDrive/DC.csv")["DC"].tolist()
p_factor = pd.read_csv("/content/drive/MyDrive/P.csv")["P"].tolist()
en_reduc = [0, 0.1, 0.2]
cost_en_reduc = [0, 100000, 200000]

In [47]:
#  creating the model

model = Model(name="microgrid", sense="MIN", solver_name=CBC)
model.emphasis = 2
model.max_seconds = 3000000

In [48]:
# adding variables to the model

# binary variables
k = 3
v = [model.add_var(name='bin_efficiency', var_type=BINARY) for i in range(k)]


# continuous non-negative variables
t = 8760

# Grid electricity purchased at time t
x = [model.add_var(name='grid_electricity', var_type=CONTINUOUS, lb=0) for j in range(t)]

#  PV electricity harnessed at time t
y = [model.add_var(name='pv_electricity', var_type=CONTINUOUS, lb=0) for j in range(t)]

# PV electricity used by the DC at time t
r = [model.add_var(name='pv_el_to_dc', var_type=CONTINUOUS, lb=0) for j in range(t)]

# PV electricity used to charge BESS at time t
h = [model.add_var(name='pv_el_to_bess', var_type=CONTINUOUS, lb=0) for j in range(t)]

# BESS electricity used by the DC at time t
b = [model.add_var(name='bess_el_to_dc', var_type=CONTINUOUS, lb=0) for j in range(t)]

# BESS storage level at time t
e = [model.add_var(name='bess_el_stored', var_type=CONTINUOUS, lb=0) for j in range(t)]

# BESS installed capacity
z = model.add_var(name='bess_capacity', var_type=CONTINUOUS, lb=0)

# PV installed capacity
w = model.add_var(name='pv_capacity', var_type=CONTINUOUS, lb=0)

In [49]:
# Constraints

for i in range(k):
    model.add_constr(xsum(v[i] for i in range(k)) == 1, name="eff_measures")

for j in range(t):
    model.add_constr((x[j] + r[j] + b[j] - dc_load[j]*(1 -xsum(en_reduc[i]*v[i] for i in range(k)))== 0), name="dc_balance")
    model.add_constr((r[j] + h[j] <= y[j]), name="pv_capacity1")
    model.add_constr((y[j] <= w*p_factor[j]*time_period), name="pv_capacity2")
    model.add_constr((e[j] -(1-D)*e[j-1]- h[j]*F + b[j] == 0), name="bess_balance")
    model.add_constr((e[j] <= z), name="bess_limits")
    model.add_constr((h[j]*F <= g_max*z), name="bess_ramping_up")
    model.add_constr((b[j] <= g_min*z), name="bess_ramping_down")

model.add_constr(w <= max_pv_cap, name="space_restr_pv")
model.add_constr(z <= max_bess_cap, name="space_restr_bess")

### Solve Model for Different Objectives

In [50]:
# Objective function

obj1 = (xsum(emission_mac*x[j] for j in range(t)) + emission_pv * w + emission_bess * z)
obj2 = xsum(cost_mac*x[j] for j in range(t)) + cost_pv * w + cost_bess * z + xsum(cost_en_reduc[i]*v[i] for i in range(k))

In [53]:
# optimization

def solve_model_obj_x(model, obj, obj_name, obj_unit):
  model.objective = obj
  status = model.optimize()

  if status == OptimizationStatus.OPTIMAL:
    print('Objective to minimise: ', obj_name)
    print('Objective unit of measure: ', obj_unit)
    print('Optimal solution found: {}'.format(round(model.objective_value, 4)))
    print("\n")

  return

In [54]:
obj_list = [(obj1, "Greenhouse-gas Emissions", "KgCo2eq"), (obj2, "Costs", "$")]

for item in obj_list:
  solve_model_obj_x(model, item[0], item[1], item[2])


Objective to minimise:  Greenhouse-gas Emissions
Objective unit of measure:  KgCo2eq
Optimal solution found: 200354.7791


Objective to minimise:  Costs
Objective unit of measure:  $
Optimal solution found: 105556.2312


